# Costruzioni dataset

In [1]:
import os
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import xml.dom.minidom as minidom

## Variabili

In [2]:
flag_shape = True
flag_shape_type = True
flag_size = True

## GEM

In [14]:
path = "GEM-data"

file_names = []
for file_name in os.listdir(path):
    dataset = []

    if os.path.isfile(os.path.join(path, file_name)):
        url = os.path.join(path, file_name)
        print(url)
        tree = ET.parse(url)
        root = tree.getroot()

        shape = ''
        shape_type = ''
        size = 0
        for entry in root.iter('entry'):
            if flag_shape:
                shape = entry.get('shape')

            if flag_shape_type:
                shape_type = entry.get('shape_type')
            
            if flag_size:
                size = entry.get('size')

            mts = []

            for modifiedtripleset in entry.iter('modifiedtripleset'):
                mtriples = ""

                triples = []
                for mtriple in modifiedtripleset.iter('mtriple'):  
                    triple = mtriple.text.split(" | ")
                    triple = (triple[0], triple[1], triple[2])
                    triples.append(triple)

                triples.sort(key=lambda x: x[1])
                for triple in triples:
                    mtriples += triple[0] + " " + triple[1] + " " + triple[2] + " "
                            
                mts.append(mtriples)

            for mt in mts:
                dataset.append([mt, '', shape, shape_type, size])

        dataset = np.array(dataset)

        dataset_pd = pd.DataFrame(dataset, columns=['triple', 'sentence', 'shape', 'shape_type', 'size'])

        #remove from filename .xml
        dataset_pd.to_csv("pre-processed-data\\" + file_name[:-4] + ".csv", index=False)

GEM-data\D2T-1-CFA_WebNLG_CounterFactual.xml
GEM-data\D2T-1-FA_WebNLG_Factual.xml
GEM-data\D2T-1-FI_WebNLG_Fictional.xml
GEM-data\D2T-2-CFA_Wikidata_CounterFactual.xml
GEM-data\D2T-2-FA_Wikidata_Factual.xml
GEM-data\D2T-2-FI_Wikidata_Fictional.xml


## WebNLG

In [15]:
dataset_types = ["train", "dev"] 
triple_numbers = ["1", "2", "3", "4", "5", "6", "7"]

for dataset_type in dataset_types:
    dataset = []

    for triple_number in triple_numbers:
        path = "WebNLG-3.0\\" + dataset_type + "\\" + triple_number + "triples"
        file_names = []
        for file_name in os.listdir(path):
            if os.path.isfile(os.path.join(path, file_name)):
                url = os.path.join(path, file_name)
                tree = ET.parse(url)
                root = tree.getroot()

                shape = ''
                shape_type = ''
                size = 1
                for entry in root.iter('entry'):
                    if flag_shape:
                        shape = entry.get('shape')

                    if flag_shape_type:
                        shape_type = entry.get('shape_type')

                    if flag_size:
                        size = entry.get('size')

                    mts = []
                    
                    for modifiedtripleset in entry.iter('modifiedtripleset'):
                        mtriples = ""

                        triples = []
                        for mtriple in modifiedtripleset.iter('mtriple'):  
                            triple = mtriple.text.split(" | ")
                            triple = (triple[0], triple[1], triple[2])
                            triples.append(triple)

                        triples.sort(key=lambda x: x[1])
                        for triple in triples:
                            mtriples += triple[0] + " " + triple[1] + " " + triple[2] + " "

                        mts.append(mtriples)

                    lexs = []
                    for lex in entry.iter('lex'):
                        lexs.append(lex.text)

                    for mt in mts:
                        for lex in lexs:
                            i += 1
                            dataset.append([mt, lex, shape, shape_type, size])

    dataset = np.array(dataset)

    dataset_pd = pd.DataFrame(dataset, columns=['triple', 'sentence', 'shape', 'shape_type', 'size'])
    dataset_pd.to_csv("pre-processed-data\\" + dataset_type + ".csv", index=False)